In [1]:
#correct the csv file generated by swing by comparing to the score
filename_csv="Gerasimov FAA.csv"
lst_players=filename_csv.split(".")[0].split(" ")
name1=lst_players[0]
name2=lst_players[1]
#set the time just before the first serve played
start=5*60+30

In [ ]:
def get_all_shots_for_point(df):
    return (df[df["Point"].astype(int)==int(point_index)]).sort_values(by='Shot')

In [2]:
#REMOVE WARM UP shots and reindex points at 1
import pandas as pd
import numpy as np

df=pd.read_csv(filename_csv)
df=df.dropna(subset=["Point"])

try:
    df["Speed (KM/H)"]=round(df["Speed (KM/H)"])
except:
    pass
#remove the warm up
df=df[df["Video Time"] > start]
#reset the point at 1 (so we don't count warmup points anymore)
first_point=df.iloc[0]["Point"]
print("Point1 was actually: "+str(first_point))

#readjust point indexes
df["Point"]=df["Point"]-first_point+1
try:
    df = df.drop("Unnamed: 0", axis=1)
except:
    pass
df.to_csv(filename_csv)
df=pd.read_csv(filename_csv)
print("Point1 is now : "+str(first_point))

Point1 was actually: 1
Point1 is now : 1


In [ ]:
#REMOVE ALL SHOTS where shot=0 

print("df before:{0}".format(len(df)))
df=df[df["Shot"]>0]
print("df after:{0}".format(len(df)))
df.rename(columns = {'Speed (KM/H)':'Speed'," Hit (x)":"hit_x"," Hit (y)":"hit_y"," Hit (z)":"hit_z"}, inplace = True)
dfP1=df[df.Player=="Player"]
print("dfP1:{0}".format(len(dfP1)))
dfP2=df[df.Player=="Opponent"]
print("dfP2:{0}".format(len(dfP2)))

In [3]:
#set the winner of the point for every shot

def set_point_winner(df, point_index: int)->int:
    """
    Returns
    1 if player1 won the point
    0 if player 2
    -1 if player 1 hit his serve out
    -2 if player 2 hit his serve out
    """
    df_all_shots_for_point = get_all_shots_for_point(df)
    df_last_shot = df_all_shots_for_point[df_all_shots_for_point["Shot"]==df_all_shots_for_point["Shot"].max()]
    last_shot_result = df_last_shot.iloc[0]["Result"]
    index_winner = -1
    str_player_last_shot = df_last_shot.iloc[0]["Player"]
    print(str_player_last_shot)
    if (df_last_shot.iloc[0]["Shot"]==1) and (last_shot_result in ["Out", "Net"]):
        if (str_player_last_shot == "Player"):
            if df_last_shot.iloc[0]["Type"]=="second_serve":
                index_winner = "0" #DF
            else:
                index_winner = "-1" #Player serve out or net
        else:
            if df_last_shot.iloc[0]["Type"]=="second_serve":
                index_winner = "1"#DF
            else:
                index_winner = "-2" #Opponent serve out or net
    elif (str_player_last_shot == "Player"):
        if (last_shot_result in ["In"]):
            index_winner = "1"
        elif (last_shot_result in ["Out", "Net"]):
            index_winner = "0"
        else:
            index_winner = "-1"
    elif (str_player_last_shot == "Opponent"):
        if (last_shot_result in ["In"]):
            index_winner = "0"
        elif (last_shot_result in ["Out", "Net"]):
            index_winner = "1"
        else:
            index_winner = "-1"
    else:
        index_winner = "-1"
    #print(str(index_winner))
    df.loc[df["Point"]==point_index,"Winner_Point"]=index_winner
    
#set_point_winner(df, 8)

In [4]:
#FOR EACH UNIQUE POINT, set point_winner for all shots of this point
for point_index in df["Point"].unique():
    set_point_winner(df, point_index)
try:
    df = df.drop("Unnamed: 0", axis=1)
except:
    pass
#SAVE FILE
df.to_csv(filename_csv)

Opponent
0
Opponent
-2
Opponent
0
Player
0
Player
1
Player
0
Player
-1
Player
0
Opponent
1
Player
0
Opponent
1
Opponent
1
Player
1
Player
1
Player
1
Opponent
0
Opponent
-2
Player
0
Opponent
0
Opponent
0
Opponent
1
Player
0
Player
-1
Opponent
1
Opponent
0
Player
0
Player
0
Player
-1
Player
0
Opponent
0
Opponent
1
Opponent
1
Opponent
1
Player
1
Opponent
1
Opponent
-2
Player
1
Opponent
1
Opponent
-2
Player
1
Opponent
1
Opponent
-2
Opponent
1
Opponent
0
Player
1
Opponent
-2
Player
0
Player
1
Opponent
-2
Opponent
1
Opponent
0
Opponent
-2
Opponent
0
Player
0
Player
1
Player
1
Opponent
0
Player
1
Opponent
0
Player
0
Player
-1
Opponent
0
Opponent
-2
Player
0
Opponent
-2
Opponent
1
Player
1
Opponent
0
Opponent
-2
Player
0
Opponent
-2
Player
1
Player
-1
Opponent
0
Player
0
Opponent
0
Player
-1
Player
0
Player
0
Player
1
Player
1
Opponent
-2
Player
0
Player
0
Opponent
1
Opponent
0
Player
0
Opponent
-2
Player
0
Opponent
0
Opponent
0
Opponent
0
Player
1
Player
-1
Opponent
1
Player
1
Player
-1
Oppon

In [5]:
print(name2 + " on SERVE, point winners:")
df[(df["Shot"]==1)&(df["Player"]=="Opponent")]["Winner_Point"].value_counts()

FAA on SERVE


0     55
1     42
-2    24
Name: Winner_Point, dtype: int64

In [6]:
print(name1 + " on SERVE, point winners:")
df[(df["Shot"]==1)&(df["Player"]=="Player")]["Winner_Point"].value_counts()

Gerasimov on SERVE, point winners:


0     50
1     41
-1    20
Name: Winner_Point, dtype: int64

In [1]:
"""
check server for each serve!
check serve (if out and nb shots > 2 then it s IN) 
synchronize time between video score and file score
for each point:
	compare point winner vs score file
	- find point scored at closest time in file (max -5, +8sec)
	- if found point and error point between both sources:
		if 1 shot only
			if result == out
				result=IN
		if 2+ shots
			correct last shot from losing player => in to OUT
			remove all next shots
	- if no error
		if some earlier shots have OUT, correct them as IN
	- if not found point (+10, -10) and shot1==out or net (only serve)
		if result==in or result==out
			correct to result=out
			remove all next shots
"""
def check_server(df, point_index, df_scores):
    """
    Check if the server is right
    If not, update all shots behind serve
    """
    df_all_shots_for_point = get_all_shots_for_point(df)
    row_serve = df_all_shots_for_point[str(df_all_shots_for_point['Shot'])=="1"]
    row_last_shot = df_all_shots_for_point[df_all_shots_for_point["Shot"]==df_all_shots_for_point["Shot"].max()]
    last_shot_result = row_last_shot.iloc[0]["Result"]
    index_winner = -1
    str_player_last_shot = row_last_shot.iloc[0]["Player"]
    if ()
    return df

#FOR EACH UNIQUE POINT, set point_winner for all shots of this point
"""df_scores = pd.read_csv(filename_csv.replace("?csv","-scoreok.csv"))
for point_index in df["Point"].unique():
    check_server(df, point_index, df_scores)"""


SyntaxError: invalid syntax (<ipython-input-1-b7f22d928ad1>, line 33)